In [13]:
import hyperspy.api as hs
import plotly.graph_objects as go

from utils.get_raw_data import locate_raw_data, get_multiple_data_arrays
from helper_files.plotting import plotly_plot
from helper_files.gaussian_fitting import gaussian, n_gaussians, fit_n_peaks_to_gaussian, area_under_peak
from helper_files.calibration import calibrate_channel_width_two_peaks, channel_to_keV
from helper_files.spectrum_dict import init_known_spectrum

emsa = locate_raw_data()

In [22]:
s.plot?

Signature:
s.plot(
    xray_lines=False,
    only_lines=('a', 'b'),
    only_one=False,
    background_windows=None,
    integration_windows=None,
    navigator='auto',
    plot_markers=True,
    autoscale='v',
    norm='auto',
    axes_manager=None,
    navigator_kwds={},
    **kwargs,
)
Docstring:
Plot the EDS spectrum. The following markers can be added

- The position of the X-ray lines and their names.
- The background windows associated with each X-ray lines. A black line
  links the left and right window with the average value in each window.

Parameters
----------
xray_lines: {False, True, 'from_elements', list of string}
    If not False, indicate the position and the name of the X-ray
    lines.
    If True, if `metadata.Sample.elements.xray_lines` contains a
    list of lines use those. If `metadata.Sample.elements.xray_lines`
    is undefined or empty or if xray_lines equals 'from_elements' and
    `metadata.Sample.elements` is defined, use the same syntax as
    `add_line`

In [24]:
%matplotlib qt

In [25]:
ga30 = get_multiple_data_arrays(filters=["GaAs_30"])[0][1]
s = hs.load(emsa[6], signal_type="EDS_SEM")
s.add_elements(["Ga", "As"])
s.plot(True, navigator='slider')


In [18]:
# Ga La, Ga Lb1, As La, As Lb1, Ga Ka, Ga Kb, As Ka, As Kb
peaks = [['Ga', 'La'], ['Ga', 'Lb1'], ['As', 'La'], ['As', 'Lb1'], ['Ga', 'Ka'], ['Ga', 'Kb'], ['As', 'Ka'], ['As', 'Kb']]
peaks_names = ['Ga L&#945;', 'Ga L&#946;<sub>1</sub>', 'As L&#945;', 'As L&#946;<sub>1</sub>', 'Ga K&#945;', 'Ga K&#946;', 'As K&#945;', 'As K&#946;'],
ga_ = hs.material.elements['Ga'].Atomic_properties.Xray_lines.as_dictionary()
as_ = hs.material.elements['As'].Atomic_properties.Xray_lines.as_dictionary()

In [11]:
peaks_keV = []
peaks_weight = []
for p in peaks:
    if p[0] == 'Ga':
        print(p[0], p[1], ga_[p[1]]['energy (keV)'])
        peaks_keV.append(ga_[p[1]]['energy (keV)'])
        peaks_weight.append(ga_[p[1]]['weight'])
    else:
        print(p[0], p[1], as_[p[1]]['energy (keV)'])
        peaks_keV.append(as_[p[1]]['energy (keV)'])
        peaks_weight.append(as_[p[1]]['weight'])
peaks_keV

Ga La 1.098
Ga Lb1 1.1249
As La 1.2819
As Lb1 1.3174
Ga Ka 9.2517
Ga Kb 10.2642
As Ka 10.5436
As Kb 11.7262


[1.098, 1.1249, 1.2819, 1.3174, 9.2517, 10.2642, 10.5436, 11.7262]

In [21]:
fig_initial = go.Figure()
channels = list(range(0, len(ga30)))  # 2048
fig_initial.add_trace(go.Scatter(x=channels, y=ga30, name="Raw Data"))



In [ ]:
fig_intitial = plotly_plot(
    x=s["channel"],
    y=s["intensity"],
    title=s["name"],
    xaxis_title="channel",
    yaxis_title="intensity",
)
fig_intitial.show()

In [ ]:
# fitting

fit_vals = fit_n_peaks_to_gaussian(
    x=s["channel"],
    y=s["intensity"],
    guessed_peaks=s["peaks_channel"],
)
s["fit_params"] = fit_vals[0]
s["fit_cov"] = fit_vals[1]
# update the channel values of the peaks
s["peaks_channel"] = s["fit_params"][1::3]
# addin the fitted gaussian to the spectrum-dictionary
s["intensity_fit"] = n_gaussians(s["channel"], *fit_vals[0])

print(f'Fitted peaks at channel: {s["peaks_channel"]}')


Fitted peaks at channel: [ 943.74174502 1045.33613489 1072.45770713]


In [ ]:
fig_fit = plotly_plot(
    s["channel"],
    s["intensity"],
    y_fit=s["intensity_fit"],
    vlines=s["peaks_channel"],
    vlines_name=s["peaks_names"],
    title=f"Gaussian fitting of {s['name']}",
)
fig_fit = plotly_plot(x=s['channel'], fig=fig_fit, fit_params=s['fit_params'], title=f"Gaussian fitting of {s['name']}")
fig_fit.show()

calib = calibrate_channel_width_two_peaks(s["peaks_channel"], s["peaks_keV"])
s["dispersion"] = calib[0]
s["offset"] = calib[1]
s["kev_calibrated"] = (s["channel"] - s["offset"]) * s["dispersion"]

The calibration factor is: 0.0099661 keV/channel, with 15.425 channels zero offset


In [ ]:
print("Peak name | Peak position (mu) [keV] | Peak amplitude (a) | FWHM (std*2*sqrt(2*ln(2))) [keV] | Area under peak")
for i in range(len(s["fit_params"]) // 3):
    try:
        peak_name = s["peaks_names"][i]
    except (
        IndexError,
        TypeError,
    ):  # if s['peaks_names'] is None or not as long as the number of peaks
        peak_name = f"peak {i}"
    mu = channel_to_keV(s, value=s['fit_params'][i*3+1])
    amp = s['fit_params']
    fwhm = channel_to_keV(s, value=s['fit_params'][i * 3 + 2] * 2 * (np.log(2) * 2)**0.5, use_offset=False)
    area = area_under_peak(s['fit_params'][i*3+1], s['fit_params'][2 + 3 * i], s['fit_params'][3 * i])
    print(f"{peak_name:<9} | {mu:<24.4f} | {amp[i*3]:<18.4f} | {fwhm:<32.4f} | {area:.6f}")


s["peaks_keV_fitted"] = channel_to_keV(s, value=s['peaks_channel'])

# for plotting Ga Ka with theoretical Ga Ka1 and Ga Ka2
vertical_lines = [s["peaks_keV_fitted"][0], 9.25174, 9.22482]
vertical_lines_names = [s["peaks_names"][0], 'Ga K&#945;<sub>1</sub>', 'Ga K&#945;<sub>2</sub>' ]

fig_calib = plotly_plot(
x=s["kev_calibrated"],
y_named=[s["intensity"], "intensity"],
vlines=vertical_lines,
vlines_name=vertical_lines_names,
title=s['name'],
xaxis_title="Energy [keV]",
start=925, #  index
stop=964, #  index
)

fig_calib.update_layout(yaxis_range=[-0.025,0.4])
fig_calib.update_layout(showlegend=False)
fig_calib.show()
fig_calib.write_html("../plots/GaAs30kV-Ga-K-lines.html")

Peak name | Peak position (mu) [keV] | Peak amplitude (a) | FWHM (std*2*sqrt(2*ln(2))) [keV] | Area under peak
Ga K&#945;</sub> | 9.2517                   | 0.3561             | 0.1622                           | 0.355143
Ga K&#946;</sub> | 10.2642                  | 0.0534             | 0.2014                           | 0.053295
As K&#945;</sub> | 10.5345                  | 0.2013             | 0.1714                           | 0.200806


$$y=x^2$$